#### 可能要改改适应度函数：从以上结果我们可以看出，这几种结果对最后的优化结果都没有什么影响。但是对其优化速度和平滑性都有影响。这也正是适应度函数选择的意义，选的好的话就可以加快优化效果。

适应度函数可以对原来的目标函数值做一些变换（比如log）——减小值之间差距过大的问题

### 算法优化

限制范围变大/变量数量变多后 原方法（or穷举法）不适用

能判断凹凸性吗——离散也不好判断凹凸性

也求不了导——离散求不了导，连续目标函数的解法都用不了

智能优化算法

### 智能算法

### 遗传算法

In [620]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt

### 参数定义

In [621]:
# 常数

v = 2  # lower limit of m
M = 10  # upper limit of m
t = 2  # lower limit of theta
N = 20  # upper limit of theta
rd = 1.2*1024#kb/s  # the downlink transmission rate
ru = 1.3*1024#kb/s  # the uplink transmission rate
O = 0.5*1024#kb  # the verification feedback size
B = 1#kb  # the transaction size
psi = 0.5  # a predefined parameter that can be defined leveraging the statistics on previous processes of block verification

In [622]:
# 随机模拟的数据

Xi = np.random.uniform(10**3*8/128, 10**6*8/128, M)  # the amount of available computational resources at verifier i 

K = np.random.uniform(5, 500)  # the required computational resources for block verification task
kappa = np.random.uniform(1, 10)  # a coefficient given by the system
q = np.random.uniform(2, 10)  # an indicator factor representing the network scale
rho = np.random.normal(100, 5, M)  # the payment from verifier i to a cloud service provider
ci = np.multiply(rho, Xi)  #  the computational cost of verifier i

In [623]:
population_size = 10

dig_theta = 5  # #theta (5个0-1变量表示theta——根据取值范围定的)
M = 10  # #0-1 z
chromosome_length = dig_theta + M  # (m=sum(z), theta[5], 0-1z[M])

### 种群初始化 

In [624]:
population = []
for i in range(population_size):
        temporary = []
        #染色体暂存器
        for j in range(chromosome_length):  # 这里的chromosome_length-4要根据m的取值范围改
            temporary.append(random.randint(0,1))
            #随机产生一个染色体,由二进制数组成
        population.append(temporary)
            #将染色体添加到种群中

In [625]:
# 初始化生成chromosome_length大小的population_size个个体的二进制基因型种群
def species_origin(population_size, chromosome_length):
    population = []
    #二维列表，包含染色体和基因
    for i in range(population_size):
        temporary = []
        #染色体暂存器
        for j in range(chromosome_length):  # 这里的chromosome_length-4要根据m的取值范围改
            temporary.append(random.randint(0,1))
            #随机产生一个染色体,由二进制数组成
        population.append(temporary)
            #将染色体添加到种群中
    
    for k in range (population_size):
        
        total = 0
        
        while sum(population[k][0:M-1]) == 0:
            
            population[k][0:M-1] = np.random.randint(0,2,M)
        
        for i in range (M,chromosome_length,1):
            total += population[k][i]*(math.pow(2,i-M))
            
        while total < t or total > N:  # 提前进行了处理，不满足2<m<10的话直接重新生成 保证生成的m还是等概率均匀分布
            population[k][M:chromosome_length] = np.random.randint(0,2,dig_theta)
            total = 0
            for j in range (M,chromosome_length,1):
                total += population[k][j]*(math.pow(2,j-M))
        
    return np.array(population)
            # 将种群返回，种群是个二维数组，个体和染色体两维

### 编码 

In [643]:
# 从二进制到十进制
# input:种群,染色体长度

def transformation(population):
    
    population = population

    output_matrix = []

    length = population.shape[1]


    for i in range(population.shape[0]):

        output_line = []
        temp1 = []
        temp2 = []
        temp3 = []
        total1 = 0
        total2 = 0

        for j in range(length):
            if j < M:
                temp1.append(population[i,j])

            else:
                total1 += population[i,j]*(math.pow(2,j-M))

        temp2.append(total1)

        temp3 = [sum(temp1)]
    
        output_line = temp3 + temp2 + temp1

        output_matrix.append(output_line)
    
    return np.array(output_matrix)

### 适应度计算

In [644]:
# beta1,2,3: weights;
# m: the number of the selected verifiiers
# theta: the number of transactions per block


# 处理目标函数的Security部分分母不能为零的问题
def  denominator0(z,Xi):
    
    maxi = []
    tem = 0
    
    for i in range(10):
        if z[i] == 0:
            tem = 0
            maxi.append(tem)
        else:
            tem = K*(1/(z[i]*Xi[i]))
            maxi.append(tem)
            
    return np.max(maxi)
        

# 目标函数
def obj_fun_GA(beta1,beta2,beta3,m,theta,z):
    
    ci_sel = z*ci
    
    L = (theta*B/rd)+denominator0(z,Xi)+(psi*theta*B*m)+(O/ru)  # Latency
    S = kappa*m**q  # Secuity     
    C = np.sum(ci_sel)/theta  # Cost
    
    L_m = (N*B/rd)+denominator0(z,Xi)+(psi*N*B*M)+(O/ru)
    S_m = kappa*M**q
    C_m = ci_sel.sum()/t
    
    u = beta1*(L/L_m)+beta2*((S_m-S)/S_m)+beta3*(C/C_m)
    return u

In [645]:
# 目标函数相当于环境 对染色体进行筛选

# 将整个族群的目标函数值整合起来
def ofunction(population):
    
    f = 0
    obj_function = []
   
    # 暂存种群中的所有的染色体(十进制)
    
    beta1 = 1/3
    beta2 = 1/3
    beta3 = 1/3
    
    population_size = population.shape[0]
    
    for i in range(population_size):
        
        m = transformation(population)[i][0]
        theta = transformation(population)[i][1]
        z = transformation(population)[i][2:2+M]
        
        f = obj_fun_GA(beta1,beta2,beta3,m,theta,z)
        
        obj_function.append(f)
        # 这里目标函数，也是适应度函数
        
    return np.array(obj_function)

###### test: 怎么看起来不太随机???

population = species_origin(population_size, chromosome_length)
transformation(population)

ofunction(population)

### 选择

##### 问题：分布很不正太，可能会有很多极大值但是绝大多数函数值都很小，概率会不会受影响？

##### 适应度函数

##### 关键 !!!——能否高效选择出优秀的个体

##### 转换后要使reverse函数中(fit_max-fit_min)大于0

In [572]:
def FIT(obj_function):
    
    Fit = np.exp(np.array(obj_function)+1)**2
    
    return Fit

## population = species_origin(population_size, chromosome_length)
ofunction(population)

np.max(ofunction(population))-np.min(ofunction(population))

FIT(ofunction(population))

np.max(FIT(ofunction(population)))-np.min(FIT(ofunction(population)))

##### 提前简化算法的函数

In [636]:
# 计算适应度和
def SUM(fitness):
    total = 0
    for i in range(len(fitness)):
        total += fitness[i]
    return total
 
# 计算适应度斐波纳挈列表，这里是为了求出累积的适应度（cdf）
def cumsum(new_fitness):
    for i in range(len(new_fitness)-1,0,-1):
        # range(start,stop,[step])
        # 倒计数
        total = 0
        j = 0
        while(j <= i):
            total += new_fitness[j]
            j += 1
            
        # 这里是为了将适应度划分成区间
        new_fitness[i] = total
        new_fitness[len(new_fitness)-1] = 1 # 概率最大是1
        
    return new_fitness

##### 选择主体函数

In [637]:
# 选择种群中个体适应度最大的个体
# def selection(population,fitness):

def reverse(fitness, population_size):
    
    rev = np.zeros(population_size)
    
    fit_max = max(fitness)
    fit_min = min(fitness)
    
#     print(fit_max-fit_min)
    
#     while (fit_max-fit_min) <= 0:
#         mutation(population,pm)
    
#     print(fit_max-fit_min)
    
    
    for i in range(population_size):
        rev[i] = (fit_max - fitness[i])/(fit_max - fit_min)
        
    return rev

def selection(population):
    
#     print(ofunction(population), population_size)
    
    fitness = reverse(ofunction(population), population_size)
    
#     print(fitness)
    
    # 单个公式暂存器
    new_fitness = []
    
    # 将所有的适应度求和
    total_fitness = SUM(fitness)
    
    # 适应度值从小到大排序
#     print(fitness)
    fitness.sort()
#     print(fitness)
    

# 这里错的 因为目标函数是越小越好
#                 # 将所有个体的适应度概率化,类似于softmax
#                 for i in range(len(fitness)):
#                     new_fitness.append(fitness[i]/total_fitness)

    # 将所有个体的适应度概率化,类似于softmax
    
    
    for i in range(len(fitness)):
        new_fitness.append(fitness[i]/total_fitness)
    
#     print(fitness, total_fitness)
#     print(new_fitness)
#     print(sum(new_fitness))

    
    # 计算适应度斐波纳挈列表，即累积的适应度(按概率大小从小到大排列最后一个元素是1)
    cumsum(new_fitness)
#     print(cumsum(new_fitness))  # print会把里面再跑一遍所以不对
    
    # 新建一个list储存存活的种群
    ms = []
    
    # 求出种群长度
    population_length = pop_len = len(population)
    
    # 根据随机数确定哪几个能存活
    # 产生种群个数的随机值
    for i in range(pop_len):
        ms.append(random.random())
    
    # 存活的种群从小到大排序
    ms.sort()
#     print(ms)
    
    # 轮盘赌方式
    fitin = 0
    newin = 0
    new_population = new_pop = population
    
    while newin < pop_len:
        if(ms[newin] < new_fitness[fitin]):
            new_pop[newin] = population[fitin]
            newin += 1
        else:
            fitin += 1

#     population1 = new_pop
    
#     return population1

    population = new_pop
    
    population = np.array(population)
    
    return population

population = species_origin(population_size, chromosome_length)
ofunction(population)

np.mean(ofunction(population))

ofunction(selection(population))

np.mean(ofunction(selection(population)))

### 交叉

In [638]:
# 针对上面的问题改成随机将第i个和第j个的染色体进行交换

# def crossover(population,pc):
# pc是概率阈值，选择单点交叉还是多点交叉，生成新的交叉个体，这里没用

def crossover(population):

    pop_len = len(population)
    
#     population1 = population
 
    i = random.randint(0,len(population)-1)
    j = random.randint(0,len(population)-1)
    
#     while i == j or i+j > (len(population)-1):
#         j = random.randint(0,len(population)-1)

    while i == j:
        j = random.randint(0,len(population)-1)
        
    # 在种群个数内随机生成单点交叉点(在染色体的第cpoint个位置发生交叉)
    cpoint = random.randint(0,len(population[0]))
        
     
    temporary1 = []
    temporary2 = []
         
    # 将tmporary1作为暂存器，暂时存放第i个染色体中的前0到cpoint个基因，
    # 然后再把第j个染色体中的后cpoint到第i个染色体中的基因个数，补充到temporary2后面
    temporary1.extend(population[i][0:cpoint])
    temporary1.extend(population[j][cpoint:len(population[i])])

    # 将tmporary2作为暂存器，暂时存放第j个染色体中的前0到cpoint个基因，
    # 然后再把第i个染色体中的后cpoint到第i个染色体中的基因个数，补充到temporary2后面
    temporary2.extend(population[j][0:cpoint])
    temporary2.extend(population[i][cpoint:len(population[i])])

    # 第i个染色体和第j个染色体基因重组/交叉完成
    population[i] = temporary1
    population[j] = temporary2
    
    while np.array(transformation(population))[i][1] > N or np.array(transformation(population))[i][1] < t:
#         population1 = population
        for i in range(pop_len-1):
        
            # 在种群个数内随机生成单点交叉点(在染色体的第cpoint个位置发生交叉)
            cpoint=random.randint(0,len(population[0]))
        
     
            temporary1 = []
            temporary2 = []
         
            # 将tmporary1作为暂存器，暂时存放第i个染色体中的前0到cpoint个基因，
            # 然后再把第j个染色体中的后cpoint到第i个染色体中的基因个数，补充到temporary2后面
            temporary1.extend(population[i][0:cpoint])
            temporary1.extend(population[j][cpoint:len(population[i])])

            # 将tmporary2作为暂存器，暂时存放第j个染色体中的前0到cpoint个基因，
            # 然后再把第i个染色体中的后cpoint到第i个染色体中的基因个数，补充到temporary2后面
            temporary2.extend(population[j][0:cpoint])
            temporary2.extend(population[i][cpoint:len(population[i])])

            # 第i个染色体和第j个染色体基因重组/交叉完成
            population[i] = temporary1
            population[j] = temporary2
        
    population = np.array(population)
    
    return population
            
#     print(i,j)
                
        
#     return population

population = species_origin(population_size, chromosome_length)
ofunction(population)

np.mean(ofunction(population))

ofunction(crossover(population))

np.mean(ofunction(crossover(population)))

### 变异

In [639]:
# def mutation(population,pm):
# pm是概率阈值

def mutation(population,pm):
    
#     population1 = population
    
    # 求出种群中所有种群/个体的个数
    px = len(population)

    # 染色体/个体中基因的个数  
    py = len(population[0])
  
    
    for i in range(px):
        
        # 如果小于阈值就变异
        if(random.random() < pm):
            # 生成0到py-1的随机数
            mpoint = random.randint(0,py-1)
            
            #将mpoint个基因进行单点随机变异，变为0或者1
            if(population[i][mpoint] == 1):
                population[i][mpoint] = 0
#                 print(i)
            else:
                population[i][mpoint] = 1
#                 print(i)
#         print(i)
        
    while np.array(transformation(population))[i][1] > 20 or np.array(transformation(population))[i][1] < 2:
#         population1 = population
        for i in range(px):
        
            # 如果小于阈值就变异
            if(random.random() < pm):
                # 生成0到py-1的随机数
                mpoint = random.randint(0,py-1)
            
                #将mpoint个基因进行单点随机变异，变为0或者1
                if(population[i][mpoint] == 1):
                    population[i][mpoint] = 0
#                     print(i)
                else:
                    population[i][mpoint] = 1
#                     print(i)

    population = np.array(population)
    
    return population

#     return population1

population = species_origin(population_size, chromosome_length)
ofunction(population)

np.mean(ofunction(population))

pm = 0.01
ofunction(mutation(population,pm))

np.mean(ofunction(mutation(population,pm)))

##### 

### 主体代码

In [640]:
# # 将每一个染色体都转化成十进制 max_value为基因最大值，为了后面画图用
# def b2d(b,max_value, chromosome_length):
#     total = 0
#     for i in range(len(b)):
#         total = total+b[i]*math.pow(2,i)
#     # 从第一位开始，每一位对2求幂，然后求和，得到十进制数？
#     total = total*max_value/(math.pow(2,chromosome_length)-1)
#     return total
 
# 寻找最好的适应度和个体
def best(population,fitness):
 
    px = population.shape[0]
    bestindividual = []
    bestfitness = fitness[0]
 
    for i in range(1,px):
   # 循环找出最大的适应度，适应度最大的也就是最好的个体
        if(fitness[i] < bestfitness):
 
            bestfitness = fitness[i]
            bestindividual = population[i]
 
    return [bestindividual, bestfitness]

In [641]:
# species_origin(population_size, chromosome_length)
# transformation(population)
# ofunction(population)
# FIT(obj_function)
# selection(population)
# crossover(population)
# mutation(population,pm)

In [649]:
population_size = 200

dig_theta = 5  # #theta (5个0-1变量表示theta——根据取值范围定的)
M = 10  # #0-1 z
chromosome_length = dig_theta + M  # (m=sum(z), theta[5], 0-1z[M])

pc = 0.6
pm = 0.1
 
results = []
Fitness = []
fitmean = []
 
population = species_origin(population_size, chromosome_length)
# 生成一个初始的种群
 
for i in range(10):  # 注意这里是迭代population_size次
    
    function0 = ofunction(population)
    
#     Fitness = FIT(function0)
    
#     best_individual,best_fitness = best(population, Fitness)
    
    results.append(best(population, function0))
    # 将最好的个体和最好的适应度保存，并将最好的个体转成十进制
    
    selection(population) # 选择
    crossover(population)  # 交配
    mutation(population, pm)  # 变异

results_f = results[1:]

# # results.sort()
# X=[]
# Y=[]

# for i in range(population_size):#500轮的结果
#     X.append(results_f[i][1])
#     Y.append(results_f[i][0])

# print(X,Y)

results_f

[[array([0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]), 0.43181277451834316],
 [array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]), 0.43147732067200617],
 [array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]), 0.43147732067200617],
 [array([0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]), 0.430522431169271],
 [array([0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1]), 0.4300826125886137],
 [array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]), 0.4300826125886137],
 [array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]), 0.43181089899822084],
 [array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1]), 0.43285768735014934],
 [array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]), 0.4328703068175888]]

In [663]:
results_f[0][0]

array([0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1])

In [667]:
X=[]
for i in range(9):
    X.append(results_f[i][0])
    
X

[array([0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]),
 array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1]),
 array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]),
 array([0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]),
 array([0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1]),
 array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]),
 array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1]),
 array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1]),
 array([0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1])]

In [669]:
transformation(np.array(X))

array([[ 5., 18.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.],
       [ 5., 19.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.],
       [ 5., 18.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.],
       [ 7., 18.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  1.,  1.],
       [ 6., 16.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  1.],
       [ 6., 18.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.],
       [ 6., 18.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.],
       [ 5., 16.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.],
       [ 6., 18.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  1.]])